### get_dummies
In this notebook i try to use the `get_dummies` function from pandas library.\
In my last notebook (Titanic Machine Learning) i used manual mapping, where i map manually classification variable in ordinal ways.

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [84]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [85]:
# replace the missing data in Age columns by using their mean
train.Age = train.Age.fillna(train.Age.mean())
test.Age = test.Age.fillna(test.Age.mean())

# replace the missing data in Embarked columns by using its mode
train.Embarked = train.Embarked.fillna(train.Embarked.mode()[0])

# replace the missing data in Fare columns by using its median
test.Fare = test.Fare.fillna(test.Fare.median())

In [86]:
# drop the cabin columns
train = train.drop('Cabin',axis=1)
test = test.drop('Cabin',axis=1)

In [52]:
print(train.isna().sum())
print('-'*20)
print(test.isna().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
--------------------
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


### Feature Engineering

#### Name Column

In [87]:
train['name_title'] = train.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
train['name_title'].value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Col               2
Major             2
Lady              1
Mme               1
Sir               1
Ms                1
Capt              1
the Countess      1
Jonkheer          1
Don               1
Name: name_title, dtype: int64

In [88]:
test['name_title'] = test.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())
test['name_title'].value_counts()

Mr        240
Miss       78
Mrs        72
Master     21
Rev         2
Col         2
Dona        1
Ms          1
Dr          1
Name: name_title, dtype: int64

In [89]:
# Drop the name column for train and test dataset
train = train.drop('Name',axis=1)
test = test.drop('Name',axis=1)

#### SibSp + Parch Column

In [90]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1

train['IsAlone'] = 0
train.loc[train['FamilySize'] == 1, 'IsAlone'] = 1

In [91]:
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

test['IsAlone'] = 0
test.loc[test['FamilySize'] == 1, 'IsAlone'] = 1

In [92]:
# drop the SibSp and Parch columns
train = train.drop(['SibSp','Parch'],axis=1)
test = test.drop(['SibSp','Parch'],axis=1)

In [24]:
train.head()

,PassengerId,Survived,Pclass,Sex,Age,Ticket,Fare,Embarked,name_title,FamilySize,IsAlone
0,1,0,3,male,22.0,A/5 21171,7.2500,S,Mr,2,0
1,2,1,1,female,38.0,PC 17599,71.2833,C,Mrs,2,0
2,3,1,3,female,26.0,STON/O2. 3101282,7.9250,S,Miss,1,1
3,4,1,1,female,35.0,113803,53.1000,S,Mrs,2,0
4,5,0,3,male,35.0,373450,8.0500,S,Mr,1,1


#### Dummies for `Train Data`

In [93]:
train_dummies = pd.get_dummies(train[['Survived','Pclass','Sex','Age','Fare','Embarked','name_title','FamilySize','IsAlone']])
train_dummies.head()

,Survived,Pclass,Age,Fare,FamilySize,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,...,name_title_Master,name_title_Miss,name_title_Mlle,name_title_Mme,name_title_Mr,name_title_Mrs,name_title_Ms,name_title_Rev,name_title_Sir,name_title_the Countess
0,0,3,22.0,7.2500,2,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
1,1,1,38.0,71.2833,2,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,1,3,26.0,7.9250,1,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,1,35.0,53.1000,2,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,3,35.0,8.0500,1,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [94]:
Pclass_dummies = pd.get_dummies(train['Pclass'], prefix='Pclass')

train_dummies1 = pd.concat([train_dummies,Pclass_dummies], axis=1).drop('Pclass', axis=1)
train_dummies1.head()

,Survived,Age,Fare,FamilySize,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,name_title_Mme,name_title_Mr,name_title_Mrs,name_title_Ms,name_title_Rev,name_title_Sir,name_title_the Countess,Pclass_1,Pclass_2,Pclass_3
0,0,22.0,7.2500,2,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,1
1,1,38.0,71.2833,2,0,1,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
2,1,26.0,7.9250,1,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,1,35.0,53.1000,2,0,1,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
4,0,35.0,8.0500,1,1,0,1,0,0,1,...,0,1,0,0,0,0,0,0,0,1


In [95]:
FamilySize_dummies = pd.get_dummies(train['FamilySize'], prefix='FamilySize')

train_dummies2 = pd.concat([train_dummies1,FamilySize_dummies], axis=1).drop('FamilySize', axis=1)
train_dummies2.head()

,Survived,Age,Fare,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,name_title_Capt,...,Pclass_3,FamilySize_1,FamilySize_2,FamilySize_3,FamilySize_4,FamilySize_5,FamilySize_6,FamilySize_7,FamilySize_8,FamilySize_11
0,0,22.0,7.2500,0,0,1,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
1,1,38.0,71.2833,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,26.0,7.9250,1,1,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
3,1,35.0,53.1000,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,0,35.0,8.0500,1,0,1,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0


In [96]:
train = train_dummies2
train.head()

,Survived,Age,Fare,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,name_title_Capt,...,Pclass_3,FamilySize_1,FamilySize_2,FamilySize_3,FamilySize_4,FamilySize_5,FamilySize_6,FamilySize_7,FamilySize_8,FamilySize_11
0,0,22.0,7.2500,0,0,1,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
1,1,38.0,71.2833,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,26.0,7.9250,1,1,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
3,1,35.0,53.1000,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,0,35.0,8.0500,1,0,1,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0


#### Dummies for `test data`

In [65]:
test.head()

,PassengerId,Pclass,Sex,Age,Ticket,Fare,Embarked,name_title,FamilySize,IsAlone
0,892,3,male,34.5,330911,7.8292,Q,Mr,1,1
1,893,3,female,47.0,363272,7.0000,S,Mrs,2,0
2,894,2,male,62.0,240276,9.6875,Q,Mr,1,1
3,895,3,male,27.0,315154,8.6625,S,Mr,1,1
4,896,3,female,22.0,3101298,12.2875,S,Mrs,3,0


In [97]:
test_dummies = pd.get_dummies(test[['Pclass','Sex','Age','Fare','Embarked','name_title','FamilySize','IsAlone']])
test_dummies.head()

,Pclass,Age,Fare,FamilySize,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,name_title_Col,name_title_Dona,name_title_Dr,name_title_Master,name_title_Miss,name_title_Mr,name_title_Mrs,name_title_Ms,name_title_Rev
0,3,34.5,7.8292,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0
1,3,47.0,7.0000,2,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0
2,2,62.0,9.6875,1,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0
3,3,27.0,8.6625,1,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0
4,3,22.0,12.2875,3,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0


In [98]:
Pclass_dummies = pd.get_dummies(test['Pclass'], prefix='Pclass')

test_dummies1 = pd.concat([test_dummies,Pclass_dummies], axis=1).drop('Pclass', axis=1)
test_dummies1.head()

,Age,Fare,FamilySize,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,name_title_Col,...,name_title_Dr,name_title_Master,name_title_Miss,name_title_Mr,name_title_Mrs,name_title_Ms,name_title_Rev,Pclass_1,Pclass_2,Pclass_3
0,34.5,7.8292,1,1,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,1
1,47.0,7.0000,2,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
2,62.0,9.6875,1,1,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
3,27.0,8.6625,1,1,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
4,22.0,12.2875,3,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1


In [99]:
FamilySize_dummies = pd.get_dummies(test['FamilySize'], prefix='FamilySize')

test_dummies2 = pd.concat([test_dummies1,FamilySize_dummies], axis=1).drop('FamilySize', axis=1)
test_dummies2.head()

,Age,Fare,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,name_title_Col,name_title_Dona,...,Pclass_3,FamilySize_1,FamilySize_2,FamilySize_3,FamilySize_4,FamilySize_5,FamilySize_6,FamilySize_7,FamilySize_8,FamilySize_11
0,34.5,7.8292,1,0,1,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,47.0,7.0000,0,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
2,62.0,9.6875,1,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,27.0,8.6625,1,0,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
4,22.0,12.2875,0,1,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0


In [100]:
test = test_dummies2
test.head()

,Age,Fare,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,name_title_Col,name_title_Dona,...,Pclass_3,FamilySize_1,FamilySize_2,FamilySize_3,FamilySize_4,FamilySize_5,FamilySize_6,FamilySize_7,FamilySize_8,FamilySize_11
0,34.5,7.8292,1,0,1,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,47.0,7.0000,0,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
2,62.0,9.6875,1,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,27.0,8.6625,1,0,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
4,22.0,12.2875,0,1,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0


### Feature Scalling

In [104]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
train_scaled = train.copy()
train_scaled[['Age','Fare']]= scale.fit_transform(train_scaled[['Age','Fare']])
train_scaled

,Survived,Age,Fare,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,name_title_Capt,...,Pclass_3,FamilySize_1,FamilySize_2,FamilySize_3,FamilySize_4,FamilySize_5,FamilySize_6,FamilySize_7,FamilySize_8,FamilySize_11
0,0,-0.592481,-0.502445,0,0,1,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
1,1,0.638789,0.786845,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,-0.284663,-0.488854,1,1,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
3,1,0.407926,0.420730,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0.407926,-0.486337,1,0,1,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,-0.207709,-0.386671,1,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
887,1,-0.823344,-0.044381,1,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
888,0,0.000000,-0.176263,0,1,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
889,1,-0.284663,-0.044381,1,0,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [102]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
test_scaled = test.copy()
test_scaled[['Age','Fare']]= scale.fit_transform(test_scaled[['Age','Fare']])
test_scaled

,Age,Fare,IsAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,name_title_Col,name_title_Dona,...,Pclass_3,FamilySize_1,FamilySize_2,FamilySize_3,FamilySize_4,FamilySize_5,FamilySize_6,FamilySize_7,FamilySize_8,FamilySize_11
0,0.334993,-0.497413,1,0,1,0,1,0,0,0,...,1,1,0,0,0,0,0,0,0,0
1,1.325530,-0.512278,0,1,0,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,0
2,2.514175,-0.464100,1,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,-0.259330,-0.482475,1,0,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
4,-0.655545,-0.417492,0,1,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.000000,-0.493455,1,0,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
414,0.691586,1.314435,1,1,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
415,0.651965,-0.507796,1,0,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0
416,0.000000,-0.493455,1,0,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,0,0


### Modelling

In [103]:
X_train = train.drop('Survived', axis=1)
y_train = train['Survived']
X_test = test
X_train.shape, y_train.shape, X_test.shape

((891, 37), (891,), (418, 29))

In [105]:
X_train_scaled = train_scaled.drop('Survived', axis=1)
y_train = train_scaled['Survived']
X_test_scaled = test_scaled
X_train.shape, y_train.shape, X_test.shape

((891, 37), (891,), (418, 29))

In [120]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn_sc = knn.score(X_train, y_train)
knn_sc

0.8327721661054994

In [119]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
knn_sc_scaled = knn.score(X_train_scaled, y_train)
knn_sc_scaled

0.8720538720538721

In [109]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)
DT_sc = tree.score(X_train, y_train)
DT_sc

0.9842873176206509

In [121]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(X_train_scaled, y_train)
DT_sc_scaled = tree.score(X_train_scaled, y_train)
DT_sc_scaled

0.9842873176206509

In [112]:
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression(max_iter = 2000)
logr.fit(X_train, y_train)
LR_sc = logr.score(X_train, y_train)
LR_sc

0.8305274971941639

In [122]:
from sklearn.linear_model import LogisticRegression
logr = LogisticRegression(max_iter = 2000)
logr.fit(X_train_scaled, y_train)
LR_sc_scaled = logr.score(X_train_scaled, y_train)
LR_sc_scaled

0.8316498316498316

In [114]:
from sklearn.svm import SVC
svc = SVC(kernel="linear")
svc.fit(X_train, y_train)
svc_sc = svc.score(X_train, y_train)
svc_sc

0.8294051627384961

In [123]:
from sklearn.svm import SVC
svc = SVC(kernel="linear")
svc.fit(X_train_scaled, y_train)
svc_sc_scaled = svc.score(X_train_scaled, y_train)
svc_sc_scaled

0.8294051627384961

In [116]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
RF_sc = random_forest.score(X_train, y_train)
RF_sc

0.9842873176206509

In [124]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train_scaled, y_train)
RF_sc_scaled = random_forest.score(X_train_scaled, y_train)
RF_sc_scaled

0.9842873176206509

In [126]:
models = pd.DataFrame({
    'Model': [ 'KNN','Decision Tree', 'Logistic Regression','Support Vector Machines',  
              'Random Forest'],
    
    'Score': [knn_sc, DT_sc, LR_sc, svc_sc, RF_sc],
    
    'Scaled_score' : [knn_sc_scaled, DT_sc_scaled, LR_sc_scaled, svc_sc_scaled, RF_sc_scaled]

})
models.sort_values(by='Score', ascending=False)

,Model,Score,Scaled_score
1,Decision Tree,0.984287,0.984287
4,Random Forest,0.984287,0.984287
0,KNN,0.832772,0.872054
2,Logistic Regression,0.831650,0.831650
3,Support Vector Machines,0.829405,0.829405


### Conclusion
I compared the score between scaled and not scaled data. Only KNN show a different results.\
Decision tree and Random forest might be a overfitting because the score is nearly 100%. 